### Data filtering and Dataset construction

In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy
from scipy import stats
import re
from collections import defaultdict
import glob
import pickle
import os
from ast import literal_eval
from tqdm import tqdm
import csv
import chart_studio.tools as tls
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Dataset paths
DATA_PATH = "/work/ada-2022-project-northamericans/Data/MovieSummaries/"
IMDB_PATH = "/work/ada-2022-project-northamericans/Data/IMDB/"
KAGGLE_PATH = "/work/ada-2022-project-northamericans/Data/Kaggle/"

# Column names for MovieSummaries dataframes
PLOT_SUMMARIES_COLS = ["id", "summary"]
CHARACTER_METADATA_COLS = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie Release Date", 
                           "Character Name", "Actor DOB", "Actor gender", "Actor height (in meters)", 
                           "Actor ethnicity (Freebase ID)", "Actor Name", "Actor age at movie release", 
                           "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
MOVIE_METADATA_COLS = ["Wikipedia Movie ID", "Freebase movie ID", "Movie name", "Movie release date", 
                       "Movie box office revenue", "Movie runtime", "Movie languages (Freebase ID:name tuples)", 
                       "Movie countries (Freebase ID:name tuples)", "Movie genres (Freebase ID:name tuples)"]

# Column names and types for IMDB dataframes
IMDB_NAME_COLS = ["nconst", "primaryName", "birthYear", "deathYear", "primaryProfession", "knownForTitles"]
IMDB_TITLE_COLS = ["tconst", "titleType", "primaryTitle", "originalTitle", "isAdult", "startYear", "endYear", 
                   "runtimeMinutes", "genres"]

# Column names for Kaggle dataframes
KAGGLE_CREDITS_COLS = ["cast", "crew", "id"]
KAGGLE_MOVIE_METADATA_COLS = ["adult", "belongs_to_collection", "Budget", "Genres", "homepage", "id", "imdb_id",
                              "original_language", "original_title", "overview", "popularity", "poster_path",
                              "production_companies", "production_countries", "release_date", "Revenue", "runtime",
                              "spoken_languages", "status", "tagline", "Movie name", "video", "vote_average", "vote_count"]

In [ ]:
# Pandas options
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
pd.set_option('display.width', 0)
pd.set_option('max_colwidth', 0)

# Initialize dataframes for each dataset
plot_summaries_df = pd.read_csv(f"{DATA_PATH}plot_summaries.txt", sep="\t", names=PLOT_SUMMARIES_COLS)
character_metadata_df = pd.read_csv(f"{DATA_PATH}character.metadata.tsv", sep="\t", names=CHARACTER_METADATA_COLS)
movie_metadata_df = pd.read_csv(f"{DATA_PATH}movie.metadata.tsv", sep="\t", names=MOVIE_METADATA_COLS)
credits_metadata_df = pd.read_csv(f"{KAGGLE_PATH}credits.csv", names=KAGGLE_CREDITS_COLS)
credits_metadata_df = credits_metadata_df.iloc[1: , :].reset_index()
kaggle_movie_metadata_df = pd.read_csv(f"{KAGGLE_PATH}movies_metadata.csv", names=KAGGLE_MOVIE_METADATA_COLS)
kaggle_movie_metadata_df = kaggle_movie_metadata_df.iloc[1: , :].reset_index()

# Print the shape of dataframes to get an idea of the size of the datasets we are working with
print("plot_summaries_df shape: ", plot_summaries_df.shape)
display(plot_summaries_df.head())
print("character_metadata_df shape: ", character_metadata_df.shape)
display(character_metadata_df.head())
print("movie_metadata_df shape: ", movie_metadata_df.shape)
display(movie_metadata_df.head())
print("credits_metadata_df shape: ", credits_metadata_df.shape)
display(credits_metadata_df.head())
print("kaggle_movie_metadata_df shape: ", kaggle_movie_metadata_df.shape)
display(kaggle_movie_metadata_df.head())


plot_summaries_df shape:  (42303, 2)
/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (15,16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

character_metadata_df shape:  (450669, 13)


,Wikipedia Movie ID,Freebase Movie ID,Movie Release Date,Character Name,Actor DOB,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor Name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


movie_metadata_df shape:  (81741, 9)


,Wikipedia Movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples)
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""Horror"", ""/m/03k9fj"": ""Adventure"", ""/m/0fdjb"": ""Supernatural"", ""/m/02kdv5l"": ""Action"", ""/m/09zvmj"": ""Space western""}"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey Mystery,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0"": ""Drama"", ""/m/0hj3n01"": ""Crime Drama""}"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""Drama""}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic thriller"", ""/m/09blyk"": ""Psychological thriller""}"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


credits_metadata_df shape:  (45476, 4)


index  ...     id
0  1      ...  862  
1  2      ...  8844 
2  3      ...  15602
3  4      ...  31357
4  5      ...  11862

[5 rows x 4 columns]

kaggle_movie_metadata_df shape:  (45466, 25)


,index,adult,belongs_to_collection,Budget,Genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,Revenue,runtime,spoken_languages,status,tagline,Movie name,video,vote_average,vote_count
0,1,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,2,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92
3,4,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34
4,5,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204

In [ ]:
character_metadata_df.dtypes

Wikipedia Movie ID                 int64  
Freebase Movie ID                  object 
Movie Release Date                 object 
Character Name                     object 
Actor DOB                          object 
Actor gender                       object 
Actor height (in meters)           float64
Actor ethnicity (Freebase ID)      object 
Actor Name                         object 
Actor age at movie release         float64
Freebase character/actor map ID    object 
Freebase character ID              object 
Freebase actor ID                  object 
dtype: object

In [ ]:
files = glob.glob("/work/ada-2022-project-northamericans/Data/moviefile_with_importance/*")
arr = []
pairs = []
for filename in files:
    with open(filename, "rb") as f:
        data = pickle.load(f)
        
        pairs += list(zip(data.index, data.importance))


In [ ]:
store = {}
for ind, importance in pairs:
    store[ind] = importance

character_metadata_df["importance"] = (character_metadata_df.index).map(lambda x: store.get(x, 0))


In [ ]:
ethnicities_mapped = pd.read_csv("ethnicities_remapped.csv", names=["freebase_id", "real_ethnicity","grouped_ethnicity"], header=None)
ethnicities_mapped

grouped_pairs = zip(ethnicities_mapped["freebase_id"], ethnicities_mapped["grouped_ethnicity"])
real_pairs = zip(ethnicities_mapped["freebase_id"], ethnicities_mapped["real_ethnicity"])
grouped_pairs_dict = dict(grouped_pairs)
real_pairs_dict = dict(real_pairs)

character_metadata_df["real_ethnicity"] = character_metadata_df["Actor ethnicity (Freebase ID)"].apply(lambda x: real_pairs_dict.get(x, np.nan))
character_metadata_df["broad_ethnicity"] = character_metadata_df["Actor ethnicity (Freebase ID)"].apply(lambda x: grouped_pairs_dict.get(x, np.nan))

In [ ]:
character_metadata_df

,Wikipedia Movie ID,Freebase Movie ID,Movie Release Date,Character Name,Actor DOB,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor Name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,importance,real_ethnicity,broad_ethnicity
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,0.000000,NaN,NaN
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,0.545455,-1,NaN
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,0.272727,African Americans,Black/African
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc,0.181818,NaN,NaN
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,0.000000,NaN,NaN
5,975900,/m/03vyhn,2001-08-24,Commander Helena Braddock,1949-05-26,F,1.727,/m/0x67,Pam Grier,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,0.000000,African Americans,Black/African
6,975900,/m/03vyhn,2001-08-24,Whitlock,1945-08-02,F,1.753,NaN,Joanna Cassidy,56.0,/m/02vd6kw,/m/0bgchmx,/m/06lj1m,0.000000,NaN,NaN
7,975900,/m/03vyhn,2001-08-24,Big Daddy Mars,NaN,M,NaN,NaN,Richard Cetrone,NaN,/m/0bgchsy,/m/0bgcht0,/m/0bgcht7,0.000000,NaN,NaN
8,975900,/m/03vyhn,2001-08-24,Michael Descanso,1971-03-20,M,1.892,NaN,Liam Waite,30.0,/m/03jqhb0,/m/0bgchs4,/m/0ks8b0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Change in diversity over time

This code analyzes data on the representation of various demographic groups in the film industry, such as gender, race, and ethnicity. By examining this data over a period of time, the code is able to track changes in diversity within the industry. This information can be useful in understanding the trends and patterns in the representation of different groups in the film industry, and can help identify areas where more progress is needed in terms of diversity and inclusion. Additionally, analyzing changes in diversity over time can provide insight into the impact of various initiatives and efforts to promote diversity within the industry, and can inform strategies for future efforts to increase representation and inclusion.

In this first demonstration, we will examine the trends in the representation of women in film casts over time, analyzing the percentage of women in various cast roles such as leading actors, supporting actors, and extras.

In [ ]:
# Get subset of the data that we need
gender_subset_df = character_metadata_df[['Movie Release Date', 'Actor gender']]
gender_subset_df = gender_subset_df.dropna()
gender_subset_df['Movie Release Year'] = gender_subset_df['Movie Release Date'].str[:4]
gender_subset_df = gender_subset_df.astype({'Movie Release Year': 'int16'})
gender_subset_df = gender_subset_df.loc[gender_subset_df['Movie Release Year'].between(1913, 2013)]

gender_subset_df['If Female'] = gender_subset_df['Actor gender'].apply(lambda x: 1 if x == 'F' else 0)
gender_subset_df['If Male'] = gender_subset_df['Actor gender'].apply(lambda x: 1 if x == 'M' else 0)

gender_grouped_df = gender_subset_df.groupby('Movie Release Year', as_index=False)['If Female', 'If Male'].sum()
gender_grouped_df["Female Percentage"] = 100*(gender_grouped_df['If Female'] / (gender_grouped_df['If Female'] + gender_grouped_df['If Male']))

fig = go.Figure()

fig.add_trace(go.Scatter(x=gender_grouped_df['Movie Release Year'], y=gender_grouped_df['Female Percentage'], mode='lines'))

fig.update_layout(xaxis_title='Year', yaxis_title='Female Percentage')
fig.update_layout(title_text='Percentage of Female Actors over Time')

fig.show()

/tmp/ipykernel_743/229764489.py:12: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Note the significant drop in female percentage of movie casts around the 1940s. This could be potentially explained by looking at the historical context. This time period was marked by significant social and cultural changes, including World War II and the emergence of the Hollywood studio system, which may have had an impact on the representation of women in film.

The following code analyzes data on the representation of various ethnic groups in the film industry, tracking changes in the number of actors from different ethnic backgrounds over time. It is important to note that the data on the representation of ethnic groups in the film industry has been categorized into broader categories for the purpose of this analysis. While we have made every effort to accurately represent the diversity of ethnicities within the film industry, this categorization process may involve some subjectivity and may not capture the full range of ethnicities represented.

In [ ]:
# analyze number of minority cast and crew members and identify potential trends
ethnicity_subset_df = character_metadata_df[['Movie Release Date', 'Actor ethnicity (Freebase ID)', 'real_ethnicity', 'broad_ethnicity']]
ethnicity_subset_df = ethnicity_subset_df.dropna()
ethnicity_subset_df['Movie Release Year'] = ethnicity_subset_df['Movie Release Date'].str[:4]
ethnicity_subset_df = ethnicity_subset_df.astype({'Movie Release Year': 'int16'})
ethnicity_subset_df = ethnicity_subset_df.loc[ethnicity_subset_df['Movie Release Year'].between(1913, 2013)]

ethnicity_grouped_df = ethnicity_subset_df.groupby(['Movie Release Year', 'broad_ethnicity'], as_index=False)
ethnicity_grouped_df = ethnicity_grouped_df.size()

# Add 0 to broad ethnicity if it doesn't exist in dataframe
years = ethnicity_grouped_df['Movie Release Year'].unique()
groups = ethnicity_grouped_df['broad_ethnicity'].unique()
for year in years:
    for group in groups:
        if ethnicity_grouped_df.loc[ethnicity_grouped_df['Movie Release Year'] == year]['broad_ethnicity'].isin([group]).any():
            continue

        ethnicity_grouped_df.loc[len(ethnicity_grouped_df)] = [year, group, 0]
        
# Add the total count for the year to determine percentages
ethnicity_grouped_df = ethnicity_grouped_df.sort_values(by=['Movie Release Year', 'size'], ascending=[True, False])
total_cast_df = ethnicity_grouped_df[['Movie Release Year', 'size']].groupby('Movie Release Year', as_index=False).sum()
total_cast_df = total_cast_df.rename(columns={'size': 'Yearly Total'})
ethnicity_grouped_df = ethnicity_grouped_df.merge(total_cast_df, on='Movie Release Year', how='left')
ethnicity_grouped_df['percentage'] = 100*(ethnicity_grouped_df['size'] / ethnicity_grouped_df['Yearly Total'])

fig = go.Figure()

for group in groups:
    fig.add_trace(go.Scatter(x=ethnicity_grouped_df.loc[ethnicity_grouped_df['broad_ethnicity'] == group]['Movie Release Year'], 
                             y=ethnicity_grouped_df.loc[ethnicity_grouped_df['broad_ethnicity'] == group]['percentage'], 
                             mode='lines',
                             name=group))

fig.update_layout(xaxis_title='Year', yaxis_title='Percentage of Broad Ethnicity')
fig.update_layout(title_text='Percentage of Broad Ethnicity in Movie Casts over Time')

fig.show()

Based on the data shown in your graph, it appears that the gap between the representation of Caucasian and minority ethnicities in the film industry has shrunk over the years. While movies are still primarily dominated by Caucasian casts, this trend suggests that there has been an increase in the representation of minority ethnicities in film casts over time, relative to the representation of Caucasian ethnicities. While this is an encouraging trend, it is important to note that there may still be disparities in the representation of different ethnicities within the industry, and further efforts may be needed to ensure that all ethnicities are equally represented in film casts.

We can now perform a one-sampled t-test to show if the representation of Caucasians in film is statistically significantly different.

In [ ]:
# Approx 1.9 billion "Caucasian" people, and 7.9 billion people, for a percentage of:
null_hypothesis = 0.24
caucasian_percents = ethnicity_grouped_df.loc[ethnicity_grouped_df['broad_ethnicity'] == 'Caucasian']['percentage'].values
t, p = stats.ttest_1samp(caucasian_percents, null_hypothesis)

print(f'T-statistic: {t:.3f}')
print(f'P-value: {p:.3f}')

if p < 0.05:
    print("There is a statistically significant difference between the mean percentage of Caucasian ethnicity \
in movie casts and the null hypothesis of the Caucasian population of the world.")
else:
    print("There is not a statistically significant difference between the mean percentage of the Caucasian ethnicity \
in movie casts and the null hypothesis value of the Caucasian population of the world.")

T-statistic: 26.282
P-value: 0.000
There is a statistically significant difference between the mean percentage of Caucasian ethnicity in movie casts and the null hypothesis of the Caucasian population of the world.


According to the one-sample t-test conducted, the mean percentage of Caucasians in movie casts across multiple time periods is statistically significantly different from the percentage of Caucasians in the population. While this is not particularly surprising, given the historical dominance of Caucasians in the entertainment industry, it is important to examine and understand these disparities in representation. However, it is worth noting that the industry has been trending towards more equal representation in terms of ethnicity in recent years, and there has been increasing recognition of the need for diversity and inclusion in media.

### Type of representation within films

Are actors from minority backgrounds equally represented in movies? This part is to investigate the importance of minority actors' roles within movies. Even if the overall trend with representation minority actors increase within the movie industry, it is not clear whether these actors get significant roles.

To analyze this, we will see how many times each actor's role is referenced within the summary, and therein associating an importance score by the percentage of importance they take up within the summary. The summary is first processed by a coreference resolution library so that different references to the same character in the summary are taken into account. Afterwards, their score is given based on the share they take compared to other characters.


In [ ]:
files = glob.glob("/work/ada-2022-project-northamericans/Data/coreferences_pickled/*")
arr = []
files = sorted(files, key=len)
arr = []
for filename in tqdm(files):
    with open(filename, "rb") as f:
        data = pickle.load(f)
        arr.append([os.path.basename(filename).split(".")[0]]+data)

grouped_df = character_metadata_df.groupby("Wikipedia Movie ID")
i = 0
movieid_to_char_df = {}
for key, item in grouped_df:
    movieid_to_char_df[str(key)]= grouped_df.get_group(key)


100%|██████████| 42303/42303 [01:28<00:00, 479.73it/s]


In [ ]:
# analyze significance of roles in relation to broad ethnicity over time 
significance_subset_df = character_metadata_df[['Movie Release Date', 'Actor ethnicity (Freebase ID)', 'real_ethnicity', 'broad_ethnicity', 'importance']]
significance_subset_df = significance_subset_df.dropna()

# filter to movies between 1913 and 2013
significance_subset_df['Movie Release Year'] = significance_subset_df['Movie Release Date'].str[:4]
significance_subset_df = significance_subset_df.astype({'Movie Release Year': 'int16'})
significance_subset_df = significance_subset_df.loc[significance_subset_df['Movie Release Year'].between(1913, 2013)]

# compute average significance score per ethnicity year over year
significance_grouped_df = significance_subset_df.groupby(['Movie Release Year', 'broad_ethnicity'], as_index=False)['importance'].mean()

# Add 0 to broad ethnicity if it doesn't exist in dataframe
years = significance_grouped_df['Movie Release Year'].unique()
groups = significance_grouped_df['broad_ethnicity'].unique()
for year in years:
    for group in groups:
        if significance_grouped_df.loc[significance_grouped_df['Movie Release Year'] == year]['broad_ethnicity'].isin([group]).any():
            continue

        significance_grouped_df.loc[len(significance_grouped_df)] = [year, group, 0]
        
# Add the total count for the year to determine percentages
significance_grouped_df = significance_grouped_df.sort_values(by=['Movie Release Year', 'importance'], ascending=[True, False])
total_cast_df = significance_grouped_df[['Movie Release Year', 'importance']].groupby('Movie Release Year', as_index=False).sum()
total_cast_df = total_cast_df.rename(columns={'importance': 'Yearly Total'})
significance_grouped_df = significance_grouped_df.merge(total_cast_df, on='Movie Release Year', how='left')
significance_grouped_df['percentage'] = significance_grouped_df['importance'] / significance_grouped_df['Yearly Total']

fig = go.Figure()

# add data to figure
for group in groups:
    fig.add_trace(go.Scatter(x=significance_grouped_df.loc[significance_grouped_df['broad_ethnicity'] == group]['Movie Release Year'], 
                             y=significance_grouped_df.loc[significance_grouped_df['broad_ethnicity'] == group]['percentage'], 
                             mode='lines',
                             name=group))

fig.update_layout(xaxis_title='Year', yaxis_title='Percentage of Appearance in Summary')
fig.update_layout(title_text='Percentage of significance share by ethnicity over time')

fig.show()

Interestingly, at the beginning of the time-series, there were no significant roles held by cast members of ethnicities other than Caucasian or Latino. However, over time, the share of significant roles becomes much more diversified with each minority group taking a more equal share of significant roles. This trend makes sense as we have also observed a general increase in diversity in cast members.

To get a deeper understanding of the share of significant roles, we also employed bootstrapping to compute an average significance score for each minority group over the entire time period and visualized the results in the below pie chart

In [ ]:
# bootstrap mean significance for each ethnicity group across all movies over time period
 
total_significance_df = character_metadata_df[['Movie Release Date', 'Actor ethnicity (Freebase ID)', 'real_ethnicity', 'broad_ethnicity', 'importance']]
total_significance_df = total_significance_df.dropna()

# filter to movies between 1913 and 2013
total_significance_df['Movie Release Year'] = total_significance_df['Movie Release Date'].str[:4]
total_significance_df = total_significance_df.astype({'Movie Release Year': 'int16'})
total_significance_df = total_significance_df.loc[total_significance_df['Movie Release Year'].between(1913, 2013)]

groups = total_significance_df['broad_ethnicity'].unique()
group_to_significance = {}
group_to_ci = {}

n_iterations = 10000
for group in groups:
    # filter df by group
    group_df = total_significance_df.loc[total_significance_df['broad_ethnicity'] == group]
    # bootstrap for group to find mean
    means = []
    for i in range(n_iterations):
        # Select bootstrap sample
        bs_df = group_df.sample(len(group_df), replace=True)
        means.append(bs_df['importance'].mean())
    group_to_significance[group] = np.mean(means)
    group_to_ci[group] = np.percentile(means, [2.5, 97.5])



# normalize each groups significance
factor = 1.0 / sum(group_to_significance.values())
for k in group_to_significance:
  group_to_significance[k] = group_to_significance[k] * factor

fig = go.Figure(data=[go.Pie(labels=list(group_to_significance.keys()), values=list(group_to_significance.values()))])
fig.show()

# print confidence intervals for each group's mean
for k, v, in group_to_ci.items():
    print(f'{k} mean significance 95% CI: {v}')

Black/African mean significance 95% CI: [0.04400008 0.04971878]
Native American/Indigenous mean significance 95% CI: [0.07710139 0.0840154 ]
Caucasian mean significance 95% CI: [0.10740498 0.11243097]
South Asian mean significance 95% CI: [0.09102008 0.10020505]
East Asian mean significance 95% CI: [0.06664806 0.07951001]
Latino mean significance 95% CI: [0.08583279 0.09424447]
Middle Eastern mean significance 95% CI: [0.09055271 0.11385254]
South East Asian mean significance 95% CI: [0.11471654 0.16075354]
Pacific Islander mean significance 95% CI: [0.0715678  0.14581974]


### Economic impact of diversity

In this section, we will analyze the economic impact of diversity in regards to the gender of the cast and crew as well as the ethnicities of the cast. What we would like to find is if there is a significant correlation between how well a film performs in the box office and how diverse the cast and crew is. To do so, we will test our null hypothesis that diversity in the cast and crew has no affect on a film's profitabiliy by performing both independent and paired t-tests to assess significance and see if there is a positive or negative statistic.

In order to quantify diversity, we decided to create different diversity scores. In regards to gender, we calculate the percentage of cast and crew members who identified as female to the total number of cast and crew. In regards to ethnicity, we calculate the percentage of cast members who do not identify as Caucasain to the total number of cast members. We then created an overall diversity score that equally weighs the gender diversity score and ethnicity diversity score of each film to compute the average of the two. Each diversity score ranges between 0 to 1, 1 being the most diverse with all female and no Caucasain cast or crew.

In [ ]:
# get essential character data
character_metadata_short_df = character_metadata_df[["Wikipedia Movie ID", "Character Name", "Actor gender", "broad_ethnicity", "Actor Name", "importance"]]
character_metadata_short_df = character_metadata_short_df.dropna()

# group character data by movie
grouped_characters_df = character_metadata_short_df.groupby('Wikipedia Movie ID').agg(lambda x: list(x)).reset_index()

# get essential kaggle movie data
kaggle_movie_metadata_short_df = kaggle_movie_metadata_df[["Movie name", "Genres", "Budget", "Revenue"]]
kaggle_movie_metadata_short_df = kaggle_movie_metadata_short_df[kaggle_movie_metadata_short_df['Budget'].apply(lambda x: str(x).isdigit())]
kaggle_movie_metadata_short_df = kaggle_movie_metadata_short_df[kaggle_movie_metadata_short_df['Revenue'].apply(lambda x: str(x).isdigit())]
kaggle_movie_metadata_short_df['Genres'] = kaggle_movie_metadata_short_df['Genres'].apply(lambda x: literal_eval(x))

kaggle_movie_metadata_short_df['Budget'] = kaggle_movie_metadata_short_df['Budget'].apply(lambda x: int(x))
kaggle_movie_metadata_short_df['Revenue'] = kaggle_movie_metadata_short_df['Revenue'].apply(lambda x: int(x))

# add crew metadata
kaggle_movie_metadata_short_df = kaggle_movie_metadata_short_df.join(credits_metadata_df["crew"].dropna().apply(lambda x: literal_eval(x)))
kaggle_movie_metadata_short_df = kaggle_movie_metadata_short_df.dropna()

# merge kaggle data with cmu data
merged_movie_metadata_df = pd.merge(kaggle_movie_metadata_short_df, movie_metadata_df, on="Movie name")

# merge movie data with character data
movie_character_metadata_df = pd.merge(merged_movie_metadata_df, grouped_characters_df, on='Wikipedia Movie ID')

# drop rows where budget/revenue is 0
movie_character_metadata_df = movie_character_metadata_df.drop(movie_character_metadata_df[movie_character_metadata_df.Budget == 0].index)
movie_character_metadata_df = movie_character_metadata_df.drop(movie_character_metadata_df[movie_character_metadata_df.Revenue == 0].index)

# add crew name, gender, job columns
def getCrewName(x):
    names = []
    for n in x:
        if n['name']:
            names.append(n['name'])
    return names
def getCrewGender(x):
    genders = []
    for g in x:
        if g['gender'] == 1:
            genders.append('F')
        elif g['gender'] == 2:
            genders.append('M')
    return genders
def getCrewDept(x):
    depts = []
    for d in x:
        if d['department']:
            depts.append(d['department'])
    return depts

movie_cast_crew_metadata_df = movie_character_metadata_df.copy()
movie_cast_crew_metadata_df['Crew names'] = movie_character_metadata_df['crew'].apply(lambda x: getCrewName(x))
movie_cast_crew_metadata_df['Crew genders'] = movie_character_metadata_df['crew'].apply(lambda x: getCrewGender(x))
movie_cast_crew_metadata_df['Crew Departments'] = movie_character_metadata_df['crew'].apply(lambda x: getCrewDept(x))

# add number of cast, crew, total
movie_cast_crew_metadata_df['Num Cast'] = movie_cast_crew_metadata_df['Actor Name'].apply(lambda x: len(x))
movie_cast_crew_metadata_df['Num Crew'] = movie_cast_crew_metadata_df['Crew names'].apply(lambda x: len(x))
def totalNum(cast, crew):
    return cast + crew
movie_cast_crew_metadata_df['Num Cast and crew'] = movie_cast_crew_metadata_df.apply(lambda x: totalNum(x['Num Cast'], x['Num Crew']), axis=1)

def genRatio(x):
    if len(x) > 0:
        fem_count = 0
        for g in x:
            if g == 'F':
                fem_count += 1
        return fem_count/len(x)
    return None
# add column: ratio of F to total CAST
movie_cast_crew_metadata_df['Gender Disparity of Cast (F:M)'] = movie_cast_crew_metadata_df['Actor gender'].apply(lambda x: genRatio(x))
# add column: ratio of F to total CREW
movie_cast_crew_metadata_df['Gender Disparity of Crew (F:M)'] = movie_cast_crew_metadata_df['Crew genders'].apply(lambda x: genRatio(x))

# add column: gender diversity score
def genDivScore(cast, crew):
    score = cast
    # some data missing on crew
    if crew is not None:
        score = 0.5*cast + 0.5*crew
    return score

movie_cast_crew_metadata_df['Overall gender diversity score'] = movie_cast_crew_metadata_df.apply(lambda x: genDivScore(x['Gender Disparity of Cast (F:M)'], x['Gender Disparity of Crew (F:M)']), axis=1)

# add column ratio of POC to total
def ethRatio(x):
    if len(x) > 0:
        eth_count = 0
        for e in x:
            if e != 'Caucasian' and e != '':
                eth_count += 1
        return eth_count/len(x)
    return None
movie_cast_crew_metadata_df['Ethnicity Disparity (POC:Caucasion)'] = movie_cast_crew_metadata_df['broad_ethnicity'].apply(lambda x: ethRatio(x))

# add column overall divserity score
def divScore(gen, eth):
    return 0.5*gen + 0.5*eth
movie_cast_crew_metadata_df['Overall diversity score'] = movie_cast_crew_metadata_df.apply(lambda x: genDivScore(x['Ethnicity Disparity (POC:Caucasion)'], x['Overall gender diversity score']), axis=1)

# add column ratio of revenue to budget
def econRatio(rev, budg):
    if rev != 0 and budg != 0:
        return rev / budg
    else:
        return None
movie_cast_crew_metadata_df['Revenue to budget ratio'] = movie_cast_crew_metadata_df.apply(lambda x: econRatio(x['Revenue'], x['Budget']), axis=1)
movie_cast_crew_metadata_df = movie_cast_crew_metadata_df.dropna()

print(movie_cast_crew_metadata_df.shape)
display(movie_cast_crew_metadata_df.head(10))


(3002, 30)


Movie name  ... Revenue to budget ratio
0   Toy Story               ...  12.451801             
1   Jumanji                 ...  4.043035              
3   Waiting to Exhale       ...  5.090760              
5   Heat                    ...  3.123947              
9   Sabrina                 ...  4.466653              
11  Sabrina                 ...  4.466653              
13  Sudden Death            ...  1.838576              
14  GoldenEye               ...  6.072311              
15  The American President  ...  1.739992              
18  Nixon                   ...  0.310949              

[10 rows x 30 columns]

In [ ]:
# Analysis

movie_cast_crew_metadata_short_df = movie_cast_crew_metadata_df[['Movie name', 'Num Cast', 'Num Crew', 'Num Cast and crew', 'Gender Disparity of Cast (F:M)', 'Gender Disparity of Crew (F:M)', 'Overall gender diversity score', 'Ethnicity Disparity (POC:Caucasion)', 'Overall diversity score', 'Revenue to budget ratio']]
merged_df = pd.merge(movie_cast_crew_metadata_short_df, movie_cast_crew_metadata_short_df, how="inner", on=["Num Cast", "Num Crew", "Num Cast and crew"], suffixes=["1", "2"])
merged_df = merged_df[[ 'Movie name1', 'Movie name2',
                        'Gender Disparity of Cast (F:M)1', 'Gender Disparity of Cast (F:M)2',
                        'Gender Disparity of Crew (F:M)1', 'Gender Disparity of Crew (F:M)2',
                        'Overall gender diversity score1', 'Overall gender diversity score2',
                        'Ethnicity Disparity (POC:Caucasion)1', 'Ethnicity Disparity (POC:Caucasion)2',
                        'Overall diversity score1', 'Overall diversity score2',
                        'Revenue to budget ratio1', 'Revenue to budget ratio2']]
merged_df = merged_df[merged_df["Revenue to budget ratio1"] > merged_df["Revenue to budget ratio2"]]
merged_df = merged_df.dropna()

################## OVERALL OVERALL ####################
merged_df['Profitable div - overall'] = merged_df.apply(lambda x: x['Overall diversity score1'] if x['Revenue to budget ratio1'] >= x['Revenue to budget ratio2'] else x['Overall gender diversity score2'], axis=1)
merged_df['Non-Profitable div - overall'] = merged_df.apply(lambda x: x['Overall diversity score2'] if x['Revenue to budget ratio1'] >= x['Revenue to budget ratio2'] else x['Overall gender diversity score1'], axis=1)
################## OVERALL GENDER ####################
merged_df['Profitable div - gender'] = merged_df.apply(lambda x: x['Overall gender diversity score1'] if x['Revenue to budget ratio1'] >= x['Revenue to budget ratio2'] else x['Overall gender diversity score2'], axis=1)
merged_df['Non-Profitable div - gender'] = merged_df.apply(lambda x: x['Overall gender diversity score2'] if x['Revenue to budget ratio1'] >= x['Revenue to budget ratio2'] else x['Overall gender diversity score1'], axis=1)
################## OVERALL ETH ####################
merged_df['Profitable div - ethnicity'] = merged_df.apply(lambda x: x['Ethnicity Disparity (POC:Caucasion)1'] if x['Revenue to budget ratio1'] >= x['Revenue to budget ratio2'] else x['Ethnicity Disparity (POC:Caucasion)2'], axis=1)
merged_df['Non-Profitable div - ethnicity'] = merged_df.apply(lambda x: x['Ethnicity Disparity (POC:Caucasion)2'] if x['Revenue to budget ratio1'] >= x['Revenue to budget ratio2'] else x['Ethnicity Disparity (POC:Caucasion)1'], axis=1)

################## OVERALL OVERALL ####################
# Independent sample t-test
print("OVERALL OVERALL: ", stats.ttest_ind(merged_df['Profitable div - overall'], merged_df['Non-Profitable div - overall']))
# Paired t-test
cross_df = merged_df.copy()
cross_df['div_diff - overall'] = cross_df['Profitable div - overall'] - cross_df['Non-Profitable div - overall']
x_diff = cross_df['div_diff - overall'].mean()
# Calculate standard error and t statistic
s_diff = cross_df['div_diff - overall'].sem()
paired_t = x_diff / s_diff
# Calculate corresponding p-value, referenced: https://docs.scipy.org/doc/scipy/tutorial/stats.html
p_val = scipy.stats.t.sf(abs(paired_t), df=len(cross_df)-1)*2
print("OVERALL OVERALL: Paired t-test statistic =", paired_t, ", pvalue =", p_val)
print("OVERALL OVERALL: Mean difference =", x_diff)

################## OVERALL GENDER ####################
# Independent sample t-test
print("OVERALL GENDER: ", stats.ttest_ind(merged_df['Profitable div - gender'], merged_df['Non-Profitable div - gender']))
# Paired t-test
cross_df['div_diff - gender'] = cross_df['Profitable div - gender'] - cross_df['Non-Profitable div - gender']
x_diff = cross_df['div_diff - gender'].mean()
# Calculate standard error and t statistic
s_diff = cross_df['div_diff - gender'].sem()
paired_t = x_diff / s_diff
# Calculate corresponding p-value, referenced: https://docs.scipy.org/doc/scipy/tutorial/stats.html
p_val = scipy.stats.t.sf(abs(paired_t), df=len(cross_df)-1)*2
print("OVERALL GENDER: Paired t-test statistic =", paired_t, ", pvalue =", p_val)
print("OVERALL GENDER: Mean difference =", x_diff)

################## OVERALL ETH ####################
# Independent sample t-test
print("OVERALL ETHNICITY: ", stats.ttest_ind(merged_df['Profitable div - ethnicity'], merged_df['Non-Profitable div - ethnicity']))
# Paired t-test
cross_df['div_diff - ethnicity'] = cross_df['Profitable div - ethnicity'] - cross_df['Non-Profitable div - ethnicity']
x_diff = cross_df['div_diff - ethnicity'].mean()
# Calculate standard error and t statistic
s_diff = cross_df['div_diff - ethnicity'].sem()
paired_t = x_diff / s_diff
# Calculate corresponding p-value, referenced: https://docs.scipy.org/doc/scipy/tutorial/stats.html
p_val = scipy.stats.t.sf(abs(paired_t), df=len(cross_df)-1)*2
print("OVERALL ETHNICITY: Paired t-test statistic =", paired_t, ", pvalue =", p_val)
print("OVERALL ETHNICITY: Mean difference =", x_diff)


OVERALL OVERALL:  Ttest_indResult(statistic=-5.217337297586662, pvalue=1.8308840767272648e-07)
OVERALL OVERALL: Paired t-test statistic = -5.267764949074156 , pvalue = 1.405914720843303e-07
OVERALL OVERALL: Mean difference = -0.014081434303650543
OVERALL GENDER:  Ttest_indResult(statistic=-7.402233055319847, pvalue=1.3851604032703372e-13)
OVERALL GENDER: Paired t-test statistic = -7.408244717081399 , pvalue = 1.3694607125081795e-13
OVERALL GENDER: Mean difference = -0.019609662795609637
OVERALL ETHNICITY:  Ttest_indResult(statistic=-1.759053764571897, pvalue=0.07858168498393178)
OVERALL ETHNICITY: Paired t-test statistic = -1.7817683338054782 , pvalue = 0.07481342400398133
OVERALL ETHNICITY: Mean difference = -0.008553205811691453


According to these results, we can see that there is a significant negative relationship between how well a film performs in the box office and how diverse the cast and crew are. In other words, we reject our null hypothesis as it is evident that more profitable films have lower diversity rates. 

We can tell because both p-values for our independent and paired t-tests for overall diversity as well as gender divserity are less than 0.05, indicating a significant relationship. The t-test statistics for these tests are also highly negative, suggesting that it is highly reliable to conclude a negative relationship between a film's profitability and its diversity rate. 

Interestingly, the p-values for ethnic diversity were both above 0.05, indicating that ethnic diversity has no significant affect on a film's profitability, however the t-test statistic is not as highly negative, suggesting that this conclusion is not as reliable.

In [ ]:
# OVERALL DIV TO PROFITS

fig = px.scatter(movie_cast_crew_metadata_df, x='Revenue to budget ratio', y='Overall diversity score')
fig.update_layout(
    title="Overall diversity over movie profits",
    yaxis_range=[-0.1,1.1],
    xaxis_range=[-0.2,6]
)
fig.show()

In [ ]:
# GENDER DIV TO PROFITS

fig = px.scatter(movie_cast_crew_metadata_df, x='Revenue to budget ratio', y='Overall gender diversity score')
fig.update_layout(
    title="Gender diversity over movie profits",
    yaxis_range=[-0.1,1.1],
    xaxis_range=[-0.2,6]
)
fig.show()

In [ ]:
# ETHN DIV TO PROFITS

fig = px.scatter(movie_cast_crew_metadata_df, x='Revenue to budget ratio', y='Ethnicity Disparity (POC:Caucasion)')
fig.update_layout(
    title="Ethnicity diversity over movie profits",
    yaxis_range=[-0.1,1.1],
    xaxis_range=[-0.2,10]
)
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f72b9652-2157-4879-b741-0567715b1f22' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>